In [2]:
from __future__ import print_function
from dolfin import *
from fenics import *

# Load mesh and subdomains
mesh = Mesh("dolfin_fine.xml.gz")
sub_domains = MeshFunction("size_t", mesh, "dolfin_fine_subdomains.xml.gz")

dims_vals = [[2, 1], [2, 2], [2, 3], [3, 2]]

for dims in dims_vals:

    # Define function spaces
    VE = VectorElement("CG", mesh.ufl_cell(), dims[0])
    QE = FiniteElement("CG", mesh.ufl_cell(), dims[1])
    VQE = MixedElement([VE, QE])
    VQ = FunctionSpace(mesh, VQE)

    # No-slip boundary condition for velocity
    noslip = Constant((0, 0))
    bc0 = DirichletBC(VQ.sub(0), noslip, sub_domains, 0)

    # Inflow boundary condition for velocity
    inflow = Expression(("100.", "0.0"), degree=1)
    bc1 = DirichletBC(VQ.sub(0), inflow, sub_domains, 1)

    # Boundary condition for pressure at outflow
    zero = Constant(0)
    bc2 = DirichletBC(VQ.sub(1), zero, sub_domains, 2)

    # Collect boundary conditions
    bcs = [bc0, bc1, bc2]

    # Define variational problem
    (u, p) = TrialFunctions(VQ)
    (v, q) = TestFunctions(VQ)
    f = Constant((0, 0))
    a = (inner(grad(u), grad(v)) - div(v)*p + q*div(u))*dx
    L = inner(f, v)*dx

    # Compute solution
    w = Function(VQ)
    solve(a == L, w, bcs)

    # Split the mixed solution using deepcopy
    # (needed for further computation on coefficient vector)
    (u, p) = w.split(True)

    # Split the mixed solution using a shallow copy
    (u, p) = w.split()

    # Save solution in VTK format
    ufile_pvd = File("results/Stokes/%d_%d_velocity.pvd" % (dims[0], dims[1]))
    ufile_pvd << u
    pfile_pvd = File("results/Stokes/%d_%d_pressure.pvd" % (dims[0], dims[1]))
    pfile_pvd << p
    

Norm of velocity coefficient vector: 82.6530298211583

Norm of pressure coefficient vector: 8170.9259320657

In [2]:
dolfin.__version__

'2017.2.0'